# Введение в анализ данных
## НИУ ВШЭ, 2019-2020 учебный год

### Домашнее задание №3

Задание выполнил(а): (впишите свои фамилию и имя)

### Общая информация

__Дата выдачи:__ 06.04.2020

__Дедлайн:__ 23:59 20.04.2020


### Оценивание и штрафы

Оценка за ДЗ вычисляется по следующей формуле:

$$
min(\text{points}, 18)  \times 10 / 18,
$$

где points — количество баллов за обязательную часть, которое вы набрали. Максимальное число баллов, которое можно получить за обязательную часть — 18, за каждые полтора балла сверху вы получите 1 бонусный балл (максимум 2). Также вы можете использовать бонусные баллы, которые накопили ранее.

За сдачу задания позже срока на итоговую оценку за задание накладывается штраф в размере 1 балл в день, но получить отрицательную оценку нельзя.

__Внимание!__ Домашнее задание выполняется самостоятельно. «Похожие» решения считаются плагиатом и все задействованные студенты (в том числе те, у кого списали) не могут получить за него больше 0 баллов.

### Формат сдачи

Загрузка файлов с решениями происходит в системе [Anytask](https://anytask.org/).

Инвайт для группы ИАД-6: rd5CNrr

Перед отправкой перезагрузите ноутбук и проверьте, что все ячейки могут быть последовательно выполнены. Ноутбук должен запускаться с использованием python 3.6+

### Подготовка данных

In [1]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
%matplotlib inline

from tqdm import tqdm

In [2]:
# Качаем датасет

!wget https://www.dropbox.com/s/tg55q9mrziroyrs/train_subset.csv

--2020-04-05 19:43:44--  https://www.dropbox.com/s/tg55q9mrziroyrs/train_subset.csv
Resolving www.dropbox.com (www.dropbox.com)... 162.125.70.1, 2620:100:6026:1::a27d:4601
Connecting to www.dropbox.com (www.dropbox.com)|162.125.70.1|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/raw/tg55q9mrziroyrs/train_subset.csv [following]
--2020-04-05 19:43:45--  https://www.dropbox.com/s/raw/tg55q9mrziroyrs/train_subset.csv
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc30efc2b4b2ace7b81f03733ea6.dl.dropboxusercontent.com/cd/0/inline/A1QDBkZeseLBbiL9HLE4WDOtvfIxuU-HeMRtUNOAYeKP9TvYd8SsdT_w1X_oW_l596zPc147W8G6IgI1PrWFqh8s4Bm35GqVZCn2qmXeoZNCMvNjqO8pGqMKWAIyNe_IItw/file# [following]
--2020-04-05 19:43:45--  https://uc30efc2b4b2ace7b81f03733ea6.dl.dropboxusercontent.com/cd/0/inline/A1QDBkZeseLBbiL9HLE4WDOtvfIxuU-HeMRtUNOAYeKP9TvYd8SsdT_w1X_oW_l596zPc147W8G6IgI1PrWFqh8s4Bm35GqVZCn

### Данные

Мы имеем дело с данными с торговой платформы Avito.
Для каждого товара представлены следующие параметры:
 - title
 - description
 - Category_name
 - Category

Имеется информация об объектах 50 классов.
Задача: по новым объектам (title, description) предсказать Category.
(Очевидно, что параметр Category_name для предсказания классов использовать нельзя)

In [2]:
data = pd.read_csv("train_subset.csv", index_col='id')

data.head()

,title,description,Category_name,Category
id,,,,
382220,Прихожая,В хорошем состоянии. Торг,Мебель и интерьер,20
397529,Кордиант 215/55/16 Летние,Кордиант 215/55/16 Летние/\n /\nАртикул: 1737l...,Запчасти и аксессуары,10
584569,Стол,"Стол, 2 рабочих места . Стол серого цвета, в д...",Мебель и интерьер,20
2513100,Комбинезон,Размер-42/44,"Одежда, обувь, аксессуары",27
1091886,Ветровка,На 2 года,Детская одежда и обувь,29


In [3]:
data.shape

(30000, 4)

In [4]:
X = data[['title', 'description']].to_numpy()
y = data['Category'].to_numpy()

del data

Сразу разделим выборку на train и test.
Никакие данные из test для обучения использовать нельзя!

In [5]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [6]:
X_train[:5]

array([['Сапоги 46 размер новые', 'Сапоги 46 размер новые'],
       ['Светильники потолочный swarovski',
        'светильники потолочные swarovski 6 штук , цена за штуку. В эксплуатации 2 года , продаются в связи со сменой интерьера в квартире'],
       ['iPhone 7 plus 128GB Red красный в наличии',
        '\xa0/\n/\n Данная цена только для подписчиков Instagram: iQmac/\n/\n Новый красный айфон 7 Plus в наличии это элегантный и мощный смартфон, который готов в полной мере раскрыть новые возможности iOS 10. Аппарат с 4-ядерным процессором А10 и 3 ГБ ОЗУ с легкостью решает самые ресурсоемкие задачи, позволяя наслаждаться быстродействием «тяжелых» приложений и игр на 5,5-дюймовом дисплее. Аппарат получил экран, как у iPad Pro, так что картинка теперь соответствует кинематографическому стандарту.'],
       ['Пион Ирис Ромашка рассада',
        'Пион куст 500 р ( более 10 шт)/\nСаженец/ корень 100р/\nРастут у нас более 70 лет/\nРозовые, бордовые и белые/\nНа фото цветы 2018г/\nП. Зубчанинов

In [7]:
y_train[:5]

array([ 27,  20,  84, 106,  27])

### Токенизация (1 балл)


Токенизация -- разбиение текста на мелкие части, которые можно обработать машинными методами.
Можно использовать разные алгоритмы токенизации.
Можете использовать WordPunctTokenizer или подобрать какой-то другой, если считаете, что он лучше подойдет для этой задачи.


In [8]:
from nltk.tokenize import WordPunctTokenizer

tokenizer = WordPunctTokenizer()

text = 'Здраствуйте. Я, Кирилл. Хотел бы чтобы вы сделали игру, 3Д-экшон суть такова...'

print("before:", text,)
print("after:", tokenizer.tokenize(text.lower()),)

before: Здраствуйте. Я, Кирилл. Хотел бы чтобы вы сделали игру, 3Д-экшон суть такова...
after: ['здраствуйте', '.', 'я', ',', 'кирилл', '.', 'хотел', 'бы', 'чтобы', 'вы', 'сделали', 'игру', ',', '3д', '-', 'экшон', 'суть', 'такова', '...']


__Задание:__ реализуйте функцию ниже

In [232]:
def preprocess(text: str) -> str:
    """
    Данная функция принимает на вход текст, 
    а возвращает тот же текст, но с пробелами между каждым токеном
    """
    
    return # your code here

In [225]:
assert preprocess(text) == 'здраствуйте . я , кирилл . хотел бы чтобы вы сделали игру , 3д - экшон суть такова ...'

__Задание:__ токенизируйте title и description в train и test

In [0]:
# Your code here

In [14]:
assert X_train[10][1] == 'продам иж планета 3 , 76 год , ( стоит на старом учёте , документы утеряны ) на ходу , хорошее состояние , все интересующие вопросы по телефону ( с родной коляской на 3 тысячи дороже ) . торга не будет .'

### BOW (3 балла)

Один из традиционных подходов -- построение bag of words.

Метод состоит в следующем:

 - Составить словарь самых часто встречающихся слов в train data
 - Для каждого примера из train посчитать, сколько раз каждое слово из словаря в нём встречается


 В sklearn есть CountVectorizer, но в этом задании его использовать нельзя.

__Задание:__ создайте словарь, где в соответствии каждому токену стоит количество раз, которое оно встретилось в X_train

In [24]:
tokens = # your code here

In [23]:
assert tokens['сапоги'] == 454

__Задание:__ выведите 10 самых частотных и 10 самых редких токенов

In [ ]:
# your code here

__Задание:__ оставьте в словаре только топ 10000 самых частотных токенов

In [ ]:
# your code here

__Задание:__ реализуйте функцию, которая предложение переводит в вектор из чисел. То есть каждому слову из словаря сопоставляется количество раз, которое оно встретилось в предложении.

In [71]:
def text_to_bow(text: str) -> np.array:
    """
    Возвращает вектор, где для каждого слова из most_common
    указано количество его употреблений
    input: строка
    output: вектор размерности словаря
    """
    
    # your code here

__Задание:__ а теперь реализуйте функцию, которая преобразует наш датасет и для каждого текста из description сопоставляет вектор.

In [73]:
def items_to_bow(items: np.array) -> np.array:
    """ Для каждого товара возвращает вектор его bow """
    # Давайте для начала попробуем строить bow только из description товара
    
    # your code here

In [75]:
X_train_bow = items_to_bow(X_train)
X_test_bow = items_to_bow(X_test)

100%|██████████| 9000/9000 [00:27<00:00, 327.69it/s]


In [271]:
# чтобы видеть проход по итерациям можно использовать библиотеку tqdm
# она работает примерно так
from tqdm import tqdm

for i in tqdm(range(100)):
    pass


100%|██████████| 100/100 [00:00<00:00, 879309.01it/s]


In [87]:
from sklearn.metrics import accuracy_score

### Логистическая регрессия и SVC (0.5 балла)


Теперь описание каждого товара представлено, как точка в многомерном пространстве.
Очень важно запомнить эту идею: дальше мы будем рассматривать разные способы перехода от текста к точке в пространстве.

Для BOW каждое измерение в пространстве -- какое-то слово.
Мы предполагаем, что текст описывается набором каких-то популярных слов, которые в нём встречаются, а близкие по смыслу тексты будут использовать одинаковые слова.

Обучите логистическую регрессию и SVC с базовыми параметрами.


In [ ]:
lr_model = 

assert accuracy_score(lr_model.predict(X_test_bow), y_test) > 0.7

In [ ]:
svc_model = 

assert accuracy_score(svc_model.predict(X_test_bow), y_test) > 0.68

### Модификация признаков (0.5 балла)

Добавьте title товара в bow с произвольным весом, как изменится качество?

In [0]:
# Your code here

Нормализуйте данные (`sklearn.preprocessing.normalize`) перед обучением. Что станет с качеством и почему?

In [0]:
# Your code here

### Mystem (1 балл)

Попробуйте обучиться, используя токенизатор mystem. Сравните качество.

In [0]:
!wget http://download.cdn.yandex.net/mystem/mystem-3.0-linux3.1-64bit.tar.gz
!tar -xvf mystem-3.0-linux3.1-64bit.tar.gz
!cp mystem /bin

In [0]:
pip install git+https://github.com/nlpub/pymystem3

In [0]:
from pymystem3 import Mystem

# Your code here

### TF-IDF (5 баллов)

Не все слова полезны одинаково, давайте попробуем [взвесить](http://tfidf.com/) их, чтобы отобрать более полезные.


> TF(t) = (Number of times term t appears in a document) / (Total number of terms in the document).
> 
> IDF(t) = log_e(Total number of documents / Number of documents with term t in it).


В sklearn есть TfidfVectorizer, но в этом задании его использовать нельзя. Для простоты посчитайте общий tf-idf для title и description (то есть каждому объекту надо сопоставить вектор, где как документ будет рассматриваться конкатенация title и description).

__Задание:__ составьте словарь, где каждому слову из изначального словаря будет стоять в соответствии количество документов, где это слово встретилось.

In [ ]:
# your code here

__Задание:__ реализуйте функцию, где тексту в соответствие ставится tf-idf вектор. 

In [163]:
def text_to_tfidf(text: str) -> np.array:
    """
    Возвращает вектор, где для каждого слова из словаря
    указан tf-idf
    """
    
    # your code here

__Задание:__ а теперь реализуйте функцию, которая преобразует наш датасет и для каждого объекта сопоставляет вектор tf-idf.

In [174]:
def items_to_tfidf(items: np.array) -> np.array:
    """ 
    Для каждого товара возвращает его tfidf вектор
    """
    
    # your code here

In [ ]:
X_train_tfidf = items_to_tfidf(X_train)
X_test_tfidf = items_to_tfidf(X_test)

__Задание:__ обучите логистическую регрессию и SVC, оцените качество (accuracy_score)

In [0]:
# Your code here

### Hashing Vectorizer (1 балл)

Попробуйте использовать `sklearn.feature_extraction.text.HashingVectorizer` для векторизации текстов.
Обязательно оцените качество работы алгоритмов классификации с использованием новой векторизации.

In [0]:
# Your code here

### Word Vectors (3 балла)

Давайте попробуем другой подход -- кажому слову сопоставим какой-то эмбеддинг (вектор).

Вектора будут небольшой размерности. Таким образом мы снизим количество параметров в модели.

Вектора мы возьмём уже готовые (обученные на текстах их интернета), так что наша модель будет знать некоторую дополнительную информацию о внешнем мире.

In [92]:
!wget https://www.dropbox.com/s/0x7oxso6x93efzj/ru.tar.gz

--2020-04-06 13:16:01--  https://www.dropbox.com/s/0x7oxso6x93efzj/ru.tar.gz
Resolving www.dropbox.com (www.dropbox.com)... 162.125.70.1, 2620:100:6026:1::a27d:4601
Connecting to www.dropbox.com (www.dropbox.com)|162.125.70.1|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/raw/0x7oxso6x93efzj/ru.tar.gz [following]
--2020-04-06 13:16:02--  https://www.dropbox.com/s/raw/0x7oxso6x93efzj/ru.tar.gz
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc928e2096207ae262b4622f28a3.dl.dropboxusercontent.com/cd/0/inline/A1Wa_ahhaEliEnHa9OB_SIFYkkJl938CPEQTCVC8R9zbAicKG8Rb59Agc0wrC3co0Vag-WrJFwxLX8r7M4RzN3J1zXIkDF1oTrlrYRUF1WiohQ/file# [following]
--2020-04-06 13:16:02--  https://uc928e2096207ae262b4622f28a3.dl.dropboxusercontent.com/cd/0/inline/A1Wa_ahhaEliEnHa9OB_SIFYkkJl938CPEQTCVC8R9zbAicKG8Rb59Agc0wrC3co0Vag-WrJFwxLX8r7M4RzN3J1zXIkDF1oTrlrYRUF1WiohQ/file
Resolving uc928e2096207ae

In [95]:
!tar -xzf ru.tar.gz

In [96]:
import gensim
from gensim.models.wrappers import FastText

model = FastText.load_fasttext_format('ru.bin')

In [102]:
# как мы видим, каждому словву данная модель сопоставляет вектор размерности 300

print(model['привет'].shape)
print(model['привет'])

(300,)
[ 0.02916384  0.02167605  0.05127367 -0.00971958  0.0465235  -0.03945766
  0.02737866  0.00638128 -0.03774629 -0.04257201 -0.00995653  0.02291315
 -0.02301722  0.06697998 -0.03674482 -0.02403202 -0.05404469  0.01372932
  0.00926399 -0.0013149   0.11941359 -0.022448    0.04011497  0.06980549
  0.00407011 -0.09384539  0.03050164 -0.02578281 -0.03525181 -0.06603175
  0.04752798  0.05874675  0.01983666  0.06092105 -0.00957561  0.08307806
 -0.01288903  0.04705157  0.02198839 -0.00649013 -0.0171444   0.03302203
  0.02124882 -0.01902875 -0.05235172  0.03458685 -0.01409259 -0.07477519
  0.01916078  0.02985001  0.0086322   0.03051201  0.02831862  0.04549561
  0.00761138 -0.05459622  0.09056009 -0.08807947 -0.05420396 -0.04793203
 -0.05672329 -0.03025264 -0.03024072 -0.05890108 -0.03137474  0.03292617
  0.05440779 -0.04548327 -0.07266086 -0.09327219  0.07247883  0.0111061
  0.01824225 -0.10570452  0.05110046 -0.04659343 -0.03277056 -0.00803401
 -0.03978698  0.00826598 -0.01074128  0.01843

In [104]:
# Будем рассматривать эмбеддинг предложения, как сумму эмбеддингов токенов

def sentence_embedding(sentence: str) -> np.array:
    """
    Складывает вектора токенов строки sentence
    """
    
    # your code here

In [267]:
assert np.allclose(sentence_embedding('сдаётся уютный , тёплый гараж для стартапов в ml')[::50],
                   np.array([ 0.08189847,  0.07249198, -0.15601222,  0.03782297,  0.09215296, -0.23092946]))

__Задание:__ сделайте все то же, что в предыдущих пунктах -- реализуйте функцию, которая преобразует данные, а затем обучите логистическую регрессию и SVM, оцените качество.

In [0]:
# a lot of your code here

### Что дальше? (6 баллов)

Для получения максимальной оценки вам нужно решить любые 2 пункта. Решение каждого пункта даст вам полтора балла:

1. Реализовать N-Gram модели текстовой классификации (__1.5 балла__)

2. Поработать с другими эмбеддингами для слов (например word2vec или GloVe) (__1.5 балла__)

3. Другие способы токенизации (pymorphy2, spaCy) (__1.5 балла__)

4. Добиться качества > 0.765 на тестовых данных (попробуйте другие токенизаторы, предобработку текста, и любые другие идеи, которые вам придут в голову) (__1.5 балла__)

Снабжайте код пояснениями и графиками.
Обязательно необходимо написать вывод по каждому пункту, который вы реализуете.

In [ ]:
# две строчки кода здесь